In [1]:
import subprocess
import json
from dotenv import load_dotenv
import os
from bit import Key, PrivateKey, PrivateKeyTestnet
from bit.network import NetworkAPI
from web3 import Web3
from eth_account import Account

In [2]:
load_dotenv()
from constants import *
w3 = Web3(Web3.HTTPProvider("http://127.0.0.1:8545"))

In [3]:
mnemonic = os.getenv('mnemonic')
# print(mnemonic)

better mystery shallow snack elbow whale abuse heart body obey silk asset


In [4]:
def derive_wallets(mnemonic, coin, numderive):
    command = f'php derive -g --mnemonic="{mnemonic}" --numderive={numderive} --coin={coin} --format=json'
    p = subprocess.Popen(command,stdout=subprocess.PIPE,shell=True)
    (output, err) = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [5]:
coins = {'btc': derive_wallets(mnemonic=mnemonic,coin=BTC,numderive=3),'eth':derive_wallets(mnemonic=mnemonic,coin=ETH,numderive=3),'btc-test': derive_wallets(mnemonic=mnemonic,coin=BTCTEST,numderive=3)}
# print(json.dumps(coins, indent=4, sort_keys=True))

{
    "btc": [
        {
            "address": "1HXE3uQTnEZUmmLtF3DwA8etvDMasTtTWf",
            "index": 0,
            "path": "m/44'/0'/0'/0/0",
            "privkey": "L5UYexkSVa9vroE2usKfzosuzJQdtqefFDcfdZkapdL1SoUKMYxU",
            "pubkey": "02a3f7407825d6cae3f360a1c29b79fb3651b8acc3d7be7855cbf9fb022322ad32",
            "pubkeyhash": "b5396291bcb0936e593009cd789a7c263a6d12c2",
            "xprv": "xprvA2xW13ZtH8XMgjT5D1DQxJhZVrR1PxcDu8pnjDQkoKcg8f2FepD2Kygisu5KsSWq7wJR3u62nxcsvHFd5nyGKbdbxacLdB3nW4A7d7X98q9",
            "xpub": "xpub6FwrQZ6n7W5euDXYK2kRKSeJ3tFVoRL5GMkPXbpNMf9f1TMQCMXGsn1Cj9UpQ6rYSFmxHaGirYdkufCiD6CwNAmTkG3dyTohoeYMjd2Pszd"
        },
        {
            "address": "1JPgRZq8J4thtoAdiCnnd3w9ps13GKuLWw",
            "index": 1,
            "path": "m/44'/0'/0'/0/1",
            "privkey": "L5D82eCQHY7kbqnJK7M4TJbUagS9ji9gL9D5Egskg8bEmwhtHMK1",
            "pubkey": "0283d6f98ca30227695e758c47f8938dfc6d90951886e0521c96a6298dc845e556",
            "pubkeyhash":

In [6]:
btc_key = coins['btc'][0]['privkey']
eth_key = coins['eth'][0]['privkey']
btctest_key = coins['btc-test'][0]['privkey']
# print(btc_key)
# print(eth_key)
# print(btctest_key)

L5UYexkSVa9vroE2usKfzosuzJQdtqefFDcfdZkapdL1SoUKMYxU
0xcb6e419dc87313939d08dccbdb0bffb10234dae025f66a992355d28cc8478dfe
cMqo5ku7h4XUWgLqEyje9hRAQ2H5LBA2bxVGnPM25CTxYUx43XXH


In [7]:
btc_recipient = coins['btc'][1]['address']
eth_recipient = coins['eth'][1]['address']
btctest_recipient = coins['btc-test'][1]['address']
# print(btc_recipient)
# print(eth_recipient)
# print(btctest_recipient)

1JPgRZq8J4thtoAdiCnnd3w9ps13GKuLWw
0xfD6E926A0018c0e4897f6e222E580998AAa09539
n2KvfSVmdaF7exQg1NrayDHUocSpaszbfb


In [8]:
def priv_key_to_account(coin, priv_key):
    global account
    if coin == 'eth':
        account=Account.privateKeyToAccount(priv_key)
    else:
        account=PrivateKeyTestnet(priv_key)
    return account

In [9]:
priv_key_to_account(ETH,eth_key)

In [10]:
eth_add = coins['eth'][0]['address']
# print(eth_add)

0x4F2489075E877429Cd0b8Aa44aE9E83309d8c1D8


In [11]:
priv_key_to_account(BTCTEST,btctest_key)

<PrivateKeyTestnet: mjpctU3pUt7zGGRTc8XY4ZXEU8xEGjNMSk>

In [12]:
def create_tx(coin,account,recipient,amount):
    global tx_data
    if coin =='eth':
        gasEstimate = w3.eth.estimateGas(
#             {"from": account.address, "to": recipient, "value": amount}
            {"from": eth_add, "to": eth_recipient, "value": amount}
        )
        tx_data= {
#             "from": account.address,
            "from": eth_add,
            "to": eth_recipient,
            "value": amount,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(eth_add),
        }
        return tx_data
    else:
        tx_data = PrivateKeyTestnet.prepare_transaction(account.address, [(btctest_recipient, amount, BTC)])
        return tx_data

In [13]:
create_tx(BTCTEST,account,btctest_recipient,0.0001)

'{"unspents":[{"amount":1459787,"confirmations":0,"script":"76a9142f37ca63227db7ee753b5f29ac278af6b57293ad88ac","txid":"72bb050e548cb1ae7d7540c6debaeedcf8cc29715a6f4e24a20009738091b764","txindex":1,"type":"p2pkh","vsize":148,"segwit":false,"sequence":4294967295}],"outputs":[["n2KvfSVmdaF7exQg1NrayDHUocSpaszbfb",10000],["mjpctU3pUt7zGGRTc8XY4ZXEU8xEGjNMSk",1426735]]}'

In [14]:
create_tx(ETH,account,eth_recipient,1)

{'from': '0x4F2489075E877429Cd0b8Aa44aE9E83309d8c1D8',
 'to': '0xfD6E926A0018c0e4897f6e222E580998AAa09539',
 'value': 1,
 'gasPrice': 20000000000,
 'gas': 21000,
 'nonce': 0}

In [15]:
def send_tx(coin, account, recipient, amount):
    if coin == ETH:
        tx = create_tx(coin,account, recipient, amount)
        signed_tx = account.sign_transaction(tx)
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        print(result.hex())
        return result.hex()
    else:
        tx_data = create_tx(coin,account,recipient,amount)
        signed = account.sign_transaction(tx_data)
        NetworkAPI.broadcast_tx_testnet(signed)
        return signed

In [16]:
send_tx(BTCTEST,account,'btctest_recipient',0.0001)

'010000000164b79180730900a2244e6f5a7129ccf8dceebadec640757daeb18c540e05bb72010000006b483045022100e897efe9e1f2213e2f0d2f00d722a5bc0785ee0937250f35aeeafd9a5c48b2b102202b9a2ab42ffff10da81dc72629bd88dabd63468e649dfa8b5853da0ec9f813f001210331887a38ac101718539349b9556fedc3e241a65b9667d622a0e1269ee16e7d6dffffffff0210270000000000001976a914e444b444223302f140bae71b1a5e4614bfdb957c88ac2fc51500000000001976a9142f37ca63227db7ee753b5f29ac278af6b57293ad88ac00000000'

In [17]:
# account

<PrivateKeyTestnet: mjpctU3pUt7zGGRTc8XY4ZXEU8xEGjNMSk>

In [18]:
# print(eth_add)
# print(eth_recipient)

0x4F2489075E877429Cd0b8Aa44aE9E83309d8c1D8
0xfD6E926A0018c0e4897f6e222E580998AAa09539


In [19]:
# send_tx(ETH,eth_add,'0xfD6E926A0018c0e4897f6e222E580998AAa09539',1)
# send_tx(ETH,account,eth_recipient,1)
# send_tx(ETH,eth_add,eth_recipient,1)
